In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
pd.options.display.float_format = '{:,.2f}'.format



,set_num,name,year,theme_id,num_parts
0,001-1,Gears,1965,1,43
1,0011-2,Town Mini-Figures,1979,67,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,0
3,0012-1,Space Mini-Figures,1979,143,12
4,0013-1,Space Mini-Figures,1979,143,12


In [5]:
df_sets = pd.read_csv('data/sets.csv')
df_themes = pd.read_csv('data/themes.csv')


In [7]:
df_sets

,set_num,name,year,theme_id,num_parts
0,001-1,Gears,1965,1,43
1,0011-2,Town Mini-Figures,1979,67,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,0
3,0012-1,Space Mini-Figures,1979,143,12
4,0013-1,Space Mini-Figures,1979,143,12
...,...,...,...,...,...
18633,XWING-1,Mini X-Wing Fighter,2019,158,60
18634,XWING-2,X-Wing Trench Run,2019,158,52
18635,YODACHRON-1,Yoda Chronicles Promotional Set,2013,158,413
18636,YTERRIER-1,Yorkshire Terrier,2018,598,0
